In [ ]:
import fsspec
import dask.distributed

In [ ]:
import xarray as xr
import fsspec

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd

ebd.set_credentials(profile='esip-qhub')

profile = 'esip-qhub'
region = 'us-west-2'
endpoint = f's3.{region}.amazonaws.com'
ebd.set_credentials(profile=profile, region=region, endpoint=endpoint)
worker_max = 10
client,cluster = ebd.start_dask_cluster(profile=profile,worker_max=worker_max, 
                                      region=region, use_existing_cluster=True,
                                      adaptive_scaling=False, wait_for_cluster=False, 
                                      environment='pangeo', worker_profile='Pangeo Worker', 
                                      propagate_env=True)

In [ ]:
url = 's3://noaa-nwm-retro-v2-zarr-pds'

In [ ]:
fs1 = fsspec.filesystem('s3', anon=True)
fs2 = fsspec.filesystem('s3', anon=False)

In [ ]:
%%time
ds = xr.open_zarr(fs1.get_mapper(url), consolidated=True)

In [ ]:
cluster

In [ ]:
ds2= ds.sel(time=('2017-01'))

In [ ]:
672/24

In [ ]:
daily_mean = ds2.resample(time='D').mean()

In [ ]:
ds2.feature_id

In [ ]:
store_url = f's3://esip-qhub/usgs/rsignell/zarr/zarr_test2'
store = fs2.get_mapper(store_url)

In [ ]:
daily_mean = daily_mean.chunk({'time':31})

In [ ]:
daily_mean.streamflow

In [ ]:
def delete_s3(url):
    fs1 = fsspec.open(url, anon=False).fs
    if fs1.exists(url):
        fs1.rm(url, recursive=True)

In [ ]:
delete_s3(store_url)

In [ ]:
%%time
a = daily_mean.to_zarr(store, compute=False, mode='w', consolidated=True)

In [ ]:
daily_mean

In [ ]:
len(dict(a.dask))

In [ ]:
30*12*50000

In [ ]:
%%time
dask.compute(a, retries=10)

In [ ]:
fs2.ls(f'{store_url}/streamflow')

In [ ]:
ds3 = xr.open_zarr(store)